In [1]:
!pip install torch

In [2]:
!pip install torchvision

In [3]:
!pip install tqdm

In [4]:
# Codeblock 1
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torchvision.models as models

import numpy as np
import matplotlib.pyplot as plt

from torch.utils.data import DataLoader
from tqdm import tqdm

# Codeblock 2
!pip install torchinfo
from torchinfo import summary


In [5]:
# Device configuration
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Transform for CIFAR-10 (RGB images)
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224, 224)),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

In [6]:
# Load CIFAR-10 dataset
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, drop_last=True)

Files already downloaded and verified
Files already downloaded and verified


In [7]:

# Define the loss function
loss_function = nn.CrossEntropyLoss()


In [8]:
# Define training and evaluation functions
def train_model(model, loss_function, optimizer, have_aux=False, epochs=5):
    train_losses, test_losses = [], []
    train_accs, test_accs = [], []
    
    for epoch in range(epochs):
        num_correct_train, num_samples_train = 0, 0
        for batch, (x_train, y_train) in tqdm(enumerate(train_loader), total=len(train_loader)):
            x_train, y_train = x_train.to(device), y_train.to(device)
            
            if have_aux:
                train_preds = model(x_train)[0] 
            else:
                train_preds = model(x_train)
            train_loss = loss_function(train_preds, y_train)
            
            with torch.no_grad():
                predicted_labels = torch.max(train_preds, dim=1)[1]
                num_correct_train += (predicted_labels == y_train).sum().item()
                num_samples_train += y_train.size(0)
            
            # Backpropagation
            optimizer.zero_grad()
            train_loss.backward()
            optimizer.step()

        train_acc = num_correct_train / num_samples_train
        test_loss, test_acc = predict_test_data(model, test_loader, loss_function)
        
        train_losses.append(train_loss.item())
        test_losses.append(test_loss)
        train_accs.append(train_acc)
        test_accs.append(test_acc)

        print(f'Epoch: {epoch + 1} | Train Loss: {train_loss.item():.3f} | Test Loss: {test_loss:.3f} | '
              f'Train Acc: {train_acc:.2f} | Test Acc: {test_acc:.2f}')
    
    return model, train_losses, test_losses, train_accs, test_accs

In [9]:
def evaluate_model(model):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for x, y in test_loader:
            x, y = x.to(device), y.to(device)
            outputs = model(x)
            preds = outputs.argmax(dim=1)
            correct += (preds == y).sum().item()
            total += y.size(0)
    return correct / total

In [10]:
def predict_test_data(model, test_loader, loss_function):
    model.eval()
    num_correct, num_samples = 0, 0
    test_loss = 0
    
    with torch.no_grad():
        for x_test, y_test in test_loader:
            x_test, y_test = x_test.to(device), y_test.to(device)
            test_preds = model(x_test)
            test_loss += loss_function(test_preds, y_test).item()
            
            predicted_labels = torch.max(test_preds, dim=1)[1]
            num_correct += (predicted_labels == y_test).sum().item()
            num_samples += y_test.size(0)
    
    model.train()
    test_acc = num_correct / num_samples
    return test_loss / len(test_loader), test_acc


In [11]:
from torchvision.models import resnet18, googlenet, vgg11, mobilenet_v3_small, alexnet
from torchvision.models import ResNet18_Weights, GoogLeNet_Weights, VGG11_Weights, MobileNet_V3_Small_Weights, AlexNet_Weights

# Load models
resnet = resnet18(weights=ResNet18_Weights.DEFAULT).to(device)
resnet.fc = nn.Linear(in_features=512, out_features=10).to(device)

# GoogLeNet with auxiliary logits enabled (use them for training)
googlenet = googlenet(weights=GoogLeNet_Weights.DEFAULT, aux_logits=True).to(device)
googlenet.fc = nn.Linear(in_features=1024, out_features=10).to(device)

vgg = vgg11(weights=VGG11_Weights.DEFAULT).to(device)
vgg.classifier[6] = nn.Linear(in_features=4096, out_features=10).to(device)

mobilenet = mobilenet_v3_small(weights=MobileNet_V3_Small_Weights.DEFAULT).to(device)
mobilenet.classifier[3] = nn.Linear(in_features=1024, out_features=10).to(device)

alexnet = alexnet(weights=AlexNet_Weights.DEFAULT).to(device)
alexnet.classifier[6] = nn.Linear(in_features=4096, out_features=10).to(device)



C:\Users\abdur\anaconda3\envs\tf-env\lib\site-packages\torchvision\models\googlenet.py:341: UserWarning: auxiliary heads in the pretrained googlenet model are NOT pretrained, so make sure to train them
  warnings.warn(


In [12]:
# Modify each model for CIFAR-10 (10 classes)
for name, model in models_dict.items():
    if "resnet" in name.lower() or "googlenet" in name.lower():
        model.fc = nn.Linear(model.fc.in_features, 10)
    elif "vgg" in name.lower() or "alexnet" in name.lower() or "mobilenet" in name.lower():
        model.classifier[-1] = nn.Linear(model.classifier[-1].in_features, 10)
    model = model.to(device)

NameError: name 'models_dict' is not defined

In [13]:
# Pastikan device didefinisikan
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Import model dan weight
from torchvision.models import resnet18, googlenet, vgg11, mobilenet_v3_small, alexnet
from torchvision.models import ResNet18_Weights, GoogLeNet_Weights, VGG11_Weights, MobileNet_V3_Small_Weights, AlexNet_Weights

# Load model dan assign ke dictionary
models_dict = {
    "resnet18": resnet18(weights=ResNet18_Weights.DEFAULT).to(device),
    "googlenet": googlenet(weights=GoogLeNet_Weights.DEFAULT, aux_logits=True).to(device),
    "vgg11": vgg11(weights=VGG11_Weights.DEFAULT).to(device),
    "mobilenet_v3_small": mobilenet_v3_small(weights=MobileNet_V3_Small_Weights.DEFAULT).to(device),
    "alexnet": alexnet(weights=AlexNet_Weights.DEFAULT).to(device),
}

# Modify setiap model untuk CIFAR-10 (10 classes)
for name, model in models_dict.items():
    if "resnet" in name.lower() or "googlenet" in name.lower():
        model.fc = nn.Linear(model.fc.in_features, 10).to(device)
    elif "vgg" in name.lower() or "alexnet" in name.lower() or "mobilenet" in name.lower():
        model.classifier[-1] = nn.Linear(model.classifier[-1].in_features, 10).to(device)

# Konfirmasi model berhasil dimodifikasi
for name, model in models_dict.items():
    print(f"{name} successfully modified for CIFAR-10!")


resnet18 successfully modified for CIFAR-10!
googlenet successfully modified for CIFAR-10!
vgg11 successfully modified for CIFAR-10!
mobilenet_v3_small successfully modified for CIFAR-10!
alexnet successfully modified for CIFAR-10!


In [14]:

# Train ResNet
resnet = models.resnet18().to(device)
resnet.fc = nn.Linear(in_features=512, out_features=10).to(device)
optim_resnet = optim.Adam(resnet.parameters(), lr=0.001)
resnet_history = train_model(resnet, loss_function, optim_resnet, have_aux=False, epochs=5)


100%|████████████████████████████████████████████████████████████████████████████████| 781/781 [36:22<00:00,  2.79s/it]


Epoch: 1 | Train Loss: 1.174 | Test Loss: 1.222 | Train Acc: 0.50 | Test Acc: 0.58


100%|████████████████████████████████████████████████████████████████████████████████| 781/781 [36:41<00:00,  2.82s/it]


Epoch: 2 | Train Loss: 0.702 | Test Loss: 0.797 | Train Acc: 0.70 | Test Acc: 0.73


100%|████████████████████████████████████████████████████████████████████████████████| 781/781 [36:45<00:00,  2.82s/it]


Epoch: 3 | Train Loss: 0.563 | Test Loss: 0.728 | Train Acc: 0.78 | Test Acc: 0.75


100%|████████████████████████████████████████████████████████████████████████████████| 781/781 [36:51<00:00,  2.83s/it]


Epoch: 4 | Train Loss: 0.377 | Test Loss: 0.677 | Train Acc: 0.83 | Test Acc: 0.77


100%|████████████████████████████████████████████████████████████████████████████████| 781/781 [37:19<00:00,  2.87s/it]


Epoch: 5 | Train Loss: 0.362 | Test Loss: 0.522 | Train Acc: 0.86 | Test Acc: 0.82


In [ ]:
# Train GoogLeNet
googlenet = models.googlenet(aux_logits=True, init_weights=True).to(device)
googlenet.fc = nn.Linear(in_features=1024, out_features=10).to(device)
optim_googlenet = optim.Adam(googlenet.parameters(), lr=0.001)
googlenet_history = train_model(googlenet, loss_function, optim_googlenet, have_aux=True, epochs=5)


100%|████████████████████████████████████████████████████████████████████████████████| 781/781 [54:28<00:00,  4.19s/it]


Epoch: 1 | Train Loss: 1.328 | Test Loss: 1.366 | Train Acc: 0.42 | Test Acc: 0.53


 92%|███████████████████████████████████████████████████████████████████████▌      | 716/781 [1:02:44<06:15,  5.77s/it]

In [ ]:

# Train VGG
vgg = models.vgg11().to(device)
vgg.classifier[6] = nn.Linear(in_features=4096, out_features=10).to(device)
optim_vgg = optim.Adam(vgg.parameters(), lr=0.001)
vgg_history = train_model(vgg, loss_function, optim_vgg, have_aux=False, epochs=5)


In [ ]:

# Train MobileNet
mobilenet = models.mobilenet_v3_small().to(device)
mobilenet.classifier[3] = nn.Linear(in_features=1024, out_features=10).to(device)
optim_mobilenet = optim.Adam(mobilenet.parameters(), lr=0.001)
mobilenet_history = train_model(mobilenet, loss_function, optim_mobilenet, have_aux=False, epochs=5)


In [ ]:

# Train AlexNet
alexnet = models.alexnet().to(device)
alexnet.classifier[6] = nn.Linear(in_features=4096, out_features=10).to(device)
optim_alexnet = optim.Adam(alexnet.parameters(), lr=0.001)
alexnet_history = train_model(alexnet, loss_function, optim_alexnet, have_aux=False, epochs=5)


In [ ]:
# Plot results
plt.figure(figsize=(10, 6))
for name in results.keys():
    plt.plot(results[name]["test_acc"], label=name)
plt.legend()
plt.xlabel("Epoch")
plt.ylabel("Test Accuracy")
plt.title("Test Accuracy Comparison")
plt.grid()
plt.show()


In [ ]:
Explanation: 
The results show that ResNet18 performs better due to its residual connections, which 
help in optimizing deeper networks by mitigating vanishing gradient issues. GoogLeNet's 
inception modules also enable it to perform well by combining multiple feature scales. 
VGG11, being a simpler architecture with deeper layers, performs moderately well, while 
MobileNetV3 is optimized for efficiency but still provides good results. AlexNet, being an 
older model, lags due to its relatively shallow and less optimized architecture. 